# Multi Factor Optimization
1. Read factor ranking from files.
2. Assign weight for each factor
3. Rank again with assigned weights for each factors
4. Get Return based on rank


# Read factor ranking from files

In [2]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [71]:
rank1 = pd.read_csv('ranks/Valuation/net profit after tax_rank.csv') 
rank2 = pd.read_csv('ranks/Growth/npat_growth_rank.csv')
rank3 = pd.read_csv('ranks/Momentum/6mmomentum_rank.csv')
monthly_ret = pd.read_csv('ranks/monthly_ret.csv')
opt_result=pd.DataFrame(columns=['weight1','weight2','cumul_return','mdd','return_mdd'])
print(opt_result)

Empty DataFrame
Columns: [weight1, weight2, cumul_return, mdd, return_mdd]
Index: []


In [39]:
def rank_opt(rank1, rank2, rank3, init, unit,opt_result):
    rank1 = rank1
    rank2 = rank2
    rank3 = rank3
    iterations = int(1/unit)
    
    counter=0
    
    for x in range(1, iterations) :
        counter2=0
        for y in range(1,iterations-counter) :
            weight1 = init + unit*counter
            weight2 = init + unit*counter2
            weight3 = 1-weight1-weight2
            counter2=counter2+1
            rank_sum = rank1*weight1 + rank2*weight2 + rank3*weight3
            rank_sum.yr_month=rank1.yr_month
            rank_rank=rank_sum.rank(axis=1, ascending=True, pct=True)
            rank_rank.yr_month=rank1.yr_month
            signal=rank_rank[rank_rank<=0.05].dropna(axis=1, how='all')
            signal[signal>0]=1
            signal.yr_month=rank1.yr_month
            #signal.to_csv('ranks/multi_factor_signal.csv')
            signal=signal.shift(1)
            mm_hold=signal.count(axis=1).max()
            profit_arr=(signal*monthly_ret[signal.columns.tolist()]).dropna(axis=1, how='all')


            profit_arr['daily_ret']=profit_arr.mean(axis=1)

            profit_arr['profit']=(profit_arr['daily_ret']+1).cumprod()
            daily_profit=profit_arr['daily_ret'].tolist()
            cumulative=profit_arr['profit'].dropna().tolist()
            #print(daily_profit)
            profit_arr['yr_month']=rank1.yr_month

            win_arr = [x for x in daily_profit if x > 0]
            loss_arr=[x for x in daily_profit if x < 0]
            win_rate = len(win_arr) / (len(loss_arr)+len(win_arr))
            dd_arr=[]
            for i in range(len(cumulative)):
                if i>0:
                    dd_arr.append((max(cumulative[:i])- cumulative[i-1])/max(cumulative[:i]))
            max_dd=max(dd_arr)
            print('weight1:{} and weight2:{} and weight3:{}'.format(weight1,weight2,weight3))
            print('CUMULATIVE PROFIT: {}'.format(round(cumulative[-1],4)))
            print('MDD: {}' .format(max_dd))
            print("mm_holding:{}------winrate:{}-----profit/MDD:{}".format(mm_hold, round(win_rate,4), round(cumulative[-1]/max_dd,4)))
            new_low ={'weight1':weight1, 'weight2':weight2, 'weight3':weight3,'cumul_return':round(cumulative[-1],4),'mdd':max_dd,'return_mdd':round(cumulative[-1]/max_dd,4)}
            opt_result = opt_result
            opt_result= opt_result.append(new_low,ignore_index=True)
            opt_result.to_csv("Ranks/opt_result.csv")
        counter=counter + 1 
    return round(cumulative[-1],4)

In [40]:
rank_opt(rank1, rank2,rank3, 0.02,0.02,opt_result)

weight1:0.02 and weight2:0.02 and weight3:0.96
CUMULATIVE PROFIT: 9.5922
MDD: 0.11170492192926912
mm_holding:16------winrate:0.6923-----profit/MDD:85.8706
weight1:0.02 and weight2:0.04 and weight3:0.94
CUMULATIVE PROFIT: 10.2209
MDD: 0.11869096857643116
mm_holding:16------winrate:0.7179-----profit/MDD:86.1137
weight1:0.02 and weight2:0.06 and weight3:0.9199999999999999
CUMULATIVE PROFIT: 9.0238
MDD: 0.12728292459959814
mm_holding:16------winrate:0.7308-----profit/MDD:70.8952
weight1:0.02 and weight2:0.08 and weight3:0.9
CUMULATIVE PROFIT: 11.0012
MDD: 0.10673017336624761
mm_holding:16------winrate:0.7179-----profit/MDD:103.0749
weight1:0.02 and weight2:0.1 and weight3:0.88
CUMULATIVE PROFIT: 10.6288
MDD: 0.10461900293217367
mm_holding:16------winrate:0.7564-----profit/MDD:101.5954
weight1:0.02 and weight2:0.12000000000000001 and weight3:0.86
CUMULATIVE PROFIT: 11.8086
MDD: 0.13307757605261863
mm_holding:16------winrate:0.7436-----profit/MDD:88.7347
weight1:0.02 and weight2:0.1399999999

weight1:0.04 and weight2:0.22 and weight3:0.74
CUMULATIVE PROFIT: 11.9748
MDD: 0.08977586976048321
mm_holding:16------winrate:0.7051-----profit/MDD:133.3857
weight1:0.04 and weight2:0.24 and weight3:0.72
CUMULATIVE PROFIT: 12.0656
MDD: 0.08977586976048317
mm_holding:16------winrate:0.7179-----profit/MDD:134.3971
weight1:0.04 and weight2:0.26 and weight3:0.7
CUMULATIVE PROFIT: 10.054
MDD: 0.13609984806975567
mm_holding:16------winrate:0.7179-----profit/MDD:73.8722
weight1:0.04 and weight2:0.28 and weight3:0.6799999999999999
CUMULATIVE PROFIT: 11.6241
MDD: 0.1360998480697557
mm_holding:16------winrate:0.7436-----profit/MDD:85.4083
weight1:0.04 and weight2:0.30000000000000004 and weight3:0.6599999999999999
CUMULATIVE PROFIT: 10.574
MDD: 0.17859260325212695
mm_holding:16------winrate:0.7436-----profit/MDD:59.2076
weight1:0.04 and weight2:0.32 and weight3:0.6399999999999999
CUMULATIVE PROFIT: 10.7622
MDD: 0.19214948770563403
mm_holding:16------winrate:0.7308-----profit/MDD:56.0094
weight1:0

weight1:0.06 and weight2:0.30000000000000004 and weight3:0.6399999999999999
CUMULATIVE PROFIT: 10.4246
MDD: 0.1787748262566663
mm_holding:16------winrate:0.7308-----profit/MDD:58.3114
weight1:0.06 and weight2:0.32 and weight3:0.6199999999999999
CUMULATIVE PROFIT: 10.0067
MDD: 0.2037924835564752
mm_holding:16------winrate:0.7051-----profit/MDD:49.1026
weight1:0.06 and weight2:0.34 and weight3:0.5999999999999999
CUMULATIVE PROFIT: 10.4288
MDD: 0.20043496661084964
mm_holding:16------winrate:0.7051-----profit/MDD:52.031
weight1:0.06 and weight2:0.36000000000000004 and weight3:0.5799999999999998
CUMULATIVE PROFIT: 10.5378
MDD: 0.2005121572824759
mm_holding:16------winrate:0.7308-----profit/MDD:52.5543
weight1:0.06 and weight2:0.38 and weight3:0.5599999999999999
CUMULATIVE PROFIT: 11.0247
MDD: 0.19375337011255625
mm_holding:16------winrate:0.7308-----profit/MDD:56.9009
weight1:0.06 and weight2:0.4 and weight3:0.5399999999999999
CUMULATIVE PROFIT: 11.463
MDD: 0.18714061557178868
mm_holding:16

weight1:0.08 and weight2:0.44 and weight3:0.48000000000000004
CUMULATIVE PROFIT: 10.5467
MDD: 0.1922223243182801
mm_holding:16------winrate:0.6795-----profit/MDD:54.8672
weight1:0.08 and weight2:0.46 and weight3:0.46
CUMULATIVE PROFIT: 11.8514
MDD: 0.19222232431828007
mm_holding:16------winrate:0.7051-----profit/MDD:61.6545
weight1:0.08 and weight2:0.48000000000000004 and weight3:0.44
CUMULATIVE PROFIT: 14.0787
MDD: 0.1814172061130466
mm_holding:16------winrate:0.7051-----profit/MDD:77.6042
weight1:0.08 and weight2:0.5 and weight3:0.42000000000000004
CUMULATIVE PROFIT: 14.6826
MDD: 0.18141720611304654
mm_holding:16------winrate:0.7308-----profit/MDD:80.9328
weight1:0.08 and weight2:0.52 and weight3:0.4
CUMULATIVE PROFIT: 14.675
MDD: 0.18141720611304674
mm_holding:16------winrate:0.7179-----profit/MDD:80.8912
weight1:0.08 and weight2:0.54 and weight3:0.38
CUMULATIVE PROFIT: 15.3385
MDD: 0.18141720611304657
mm_holding:16------winrate:0.7051-----profit/MDD:84.5481
weight1:0.08 and weight2

weight1:0.1 and weight2:0.62 and weight3:0.28
CUMULATIVE PROFIT: 13.4478
MDD: 0.16598438221326015
mm_holding:16------winrate:0.7179-----profit/MDD:81.0183
weight1:0.1 and weight2:0.64 and weight3:0.26
CUMULATIVE PROFIT: 13.1398
MDD: 0.16598438221326012
mm_holding:16------winrate:0.7051-----profit/MDD:79.1627
weight1:0.1 and weight2:0.66 and weight3:0.24
CUMULATIVE PROFIT: 13.9363
MDD: 0.16225551614262199
mm_holding:16------winrate:0.7051-----profit/MDD:85.8908
weight1:0.1 and weight2:0.68 and weight3:0.21999999999999997
CUMULATIVE PROFIT: 14.609
MDD: 0.16225551614262196
mm_holding:16------winrate:0.7179-----profit/MDD:90.037
weight1:0.1 and weight2:0.7000000000000001 and weight3:0.19999999999999996
CUMULATIVE PROFIT: 14.8863
MDD: 0.16225551614262207
mm_holding:16------winrate:0.7308-----profit/MDD:91.7463
weight1:0.1 and weight2:0.7200000000000001 and weight3:0.17999999999999994
CUMULATIVE PROFIT: 13.7061
MDD: 0.16430458412507806
mm_holding:16------winrate:0.7179-----profit/MDD:83.4191

...

weight1:0.62 and weight2:0.28 and weight3:0.09999999999999998
CUMULATIVE PROFIT: 19.3351
MDD: 0.188531928725011
mm_holding:16------winrate:0.7051-----profit/MDD:102.5563
weight1:0.62 and weight2:0.30000000000000004 and weight3:0.07999999999999996
CUMULATIVE PROFIT: 19.5678
MDD: 0.18140389551939154
mm_holding:16------winrate:0.7051-----profit/MDD:107.8686
weight1:0.62 and weight2:0.32 and weight3:0.06
CUMULATIVE PROFIT: 19.6835
MDD: 0.18140389551939148
mm_holding:16------winrate:0.6923-----profit/MDD:108.5064
weight1:0.62 and weight2:0.34 and weight3:0.03999999999999998
CUMULATIVE PROFIT: 19.4693
MDD: 0.17523302365084528
mm_holding:16------winrate:0.6795-----profit/MDD:111.1054
weight1:0.62 and weight2:0.36000000000000004 and weight3:0.019999999999999962
CUMULATIVE PROFIT: 19.6948
MDD: 0.18267493290684364
mm_holding:16------winrate:0.7051-----profit/MDD:107.8131
weight1:0.62 and weight2:0.38 and weight3:0.0
CUMULATIVE PROFIT: 16.43
MDD: 0.18267493290684353
mm_holding:16------winrate:0.7

weight1:0.68 and weight2:0.28 and weight3:0.039999999999999925
CUMULATIVE PROFIT: 20.2421
MDD: 0.17032485827893828
mm_holding:16------winrate:0.6923-----profit/MDD:118.8442
weight1:0.68 and weight2:0.30000000000000004 and weight3:0.019999999999999907
CUMULATIVE PROFIT: 21.5687
MDD: 0.16595861163458472
mm_holding:16------winrate:0.6923-----profit/MDD:129.964
weight1:0.68 and weight2:0.32 and weight3:-5.551115123125783e-17
CUMULATIVE PROFIT: 19.7164
MDD: 0.1855893936738466
mm_holding:16------winrate:0.6795-----profit/MDD:106.2364
weight1:0.7000000000000001 and weight2:0.02 and weight3:0.2799999999999999
CUMULATIVE PROFIT: 8.6463
MDD: 0.2074924145502929
mm_holding:16------winrate:0.6667-----profit/MDD:41.6703
weight1:0.7000000000000001 and weight2:0.04 and weight3:0.25999999999999995
CUMULATIVE PROFIT: 9.9718
MDD: 0.17024243890315158
mm_holding:16------winrate:0.6795-----profit/MDD:58.5743
weight1:0.7000000000000001 and weight2:0.06 and weight3:0.23999999999999994
CUMULATIVE PROFIT: 13.16

weight1:0.76 and weight2:0.13999999999999999 and weight3:0.1
CUMULATIVE PROFIT: 27.1616
MDD: 0.15711211780346068
mm_holding:16------winrate:0.641-----profit/MDD:172.8801
weight1:0.76 and weight2:0.16 and weight3:0.07999999999999999
CUMULATIVE PROFIT: 23.2067
MDD: 0.16466821147897154
mm_holding:16------winrate:0.7051-----profit/MDD:140.9299
weight1:0.76 and weight2:0.18 and weight3:0.06
CUMULATIVE PROFIT: 22.1687
MDD: 0.1695612227004123
mm_holding:16------winrate:0.6923-----profit/MDD:130.7417
weight1:0.76 and weight2:0.19999999999999998 and weight3:0.04000000000000001
CUMULATIVE PROFIT: 20.5969
MDD: 0.1528046207300272
mm_holding:16------winrate:0.6923-----profit/MDD:134.7927
weight1:0.76 and weight2:0.22 and weight3:0.01999999999999999
CUMULATIVE PROFIT: 20.9368
MDD: 0.14727911476516153
mm_holding:16------winrate:0.6667-----profit/MDD:142.1575
weight1:0.76 and weight2:0.24 and weight3:0.0
CUMULATIVE PROFIT: 22.9351
MDD: 0.1535258085505114
mm_holding:16------winrate:0.6795-----profit/MD

weight1:0.86 and weight2:0.12000000000000001 and weight3:0.020000000000000004
CUMULATIVE PROFIT: 25.2504
MDD: 0.13320370402776108
mm_holding:16------winrate:0.7051-----profit/MDD:189.5622
weight1:0.86 and weight2:0.13999999999999999 and weight3:2.7755575615628914e-17
CUMULATIVE PROFIT: 23.9053
MDD: 0.13762862190525985
mm_holding:16------winrate:0.6923-----profit/MDD:173.6941
weight1:0.88 and weight2:0.02 and weight3:0.09999999999999999
CUMULATIVE PROFIT: 13.3151
MDD: 0.17166841597143356
mm_holding:16------winrate:0.641-----profit/MDD:77.5631
weight1:0.88 and weight2:0.04 and weight3:0.07999999999999999
CUMULATIVE PROFIT: 16.1114
MDD: 0.13372200478752805
mm_holding:16------winrate:0.6667-----profit/MDD:120.4846
weight1:0.88 and weight2:0.06 and weight3:0.06
CUMULATIVE PROFIT: 20.3065
MDD: 0.1216065497302352
mm_holding:16------winrate:0.6923-----profit/MDD:166.9854
weight1:0.88 and weight2:0.08 and weight3:0.039999999999999994
CUMULATIVE PROFIT: 18.1203
MDD: 0.12910930276440277
mm_holdin

18.7616

In [41]:
opt_result = pd.read_csv('ranks/opt_result.csv') 

In [42]:
print(opt_result)

      Unnamed: 0  weight1  weight2  cumul_return       mdd  return_mdd  \
0              0     0.02     0.02        9.5922  0.111705     85.8706   
1              1     0.02     0.04       10.2209  0.118691     86.1137   
2              2     0.02     0.06        9.0238  0.127283     70.8952   
3              3     0.02     0.08       11.0012  0.106730    103.0749   
4              4     0.02     0.10       10.6288  0.104619    101.5954   
...          ...      ...      ...           ...       ...         ...   
1220        1220     0.94     0.04       19.4346  0.121522    159.9266   
1221        1221     0.94     0.06       19.2224  0.131391    146.2992   
1222        1222     0.96     0.02       20.2258  0.137080    147.5477   
1223        1223     0.96     0.04       20.6471  0.127445    162.0073   
1224        1224     0.98     0.02       18.7616  0.140630    133.4114   

           weight3  
0     9.600000e-01  
1     9.400000e-01  
2     9.200000e-01  
3     9.000000e-01  
4     

In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=opt_result['weight1'].tolist(), y=opt_result['cumul_return'].tolist(),
                                 mode='lines',
                                 name='cumulative'))

fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=opt_result['weight1'].tolist(), y=opt_result['return_mdd'].tolist(),
                                 mode='lines',
                                 name='cumulative'))

fig.show()

In [73]:
weight1 = 1
weight2 = 0.0
weight3 = 1-weight1-weight2

rank_sum = rank1*weight1 + rank2*weight2 + rank3*weight3
rank_sum.yr_month=rank1.yr_month
rank_rank=rank_sum.rank(axis=1, ascending=True, pct=True)
rank_rank.yr_month=rank1.yr_month
signal=rank_rank[rank_rank<=0.03].dropna(axis=1, how='all')
signal[signal>0]=1
signal.yr_month=rank1.yr_month
signal.to_csv('ranks/multi_3factor_signal.csv')
signal=signal.shift(1)
mm_hold=signal.count(axis=1).max()
profit_arr=(signal*monthly_ret[signal.columns.tolist()]).dropna(axis=1, how='all')

signal.to_csv('equity_netprofitgrowth_signal.csv')
profit_arr['daily_ret']=profit_arr.mean(axis=1)

profit_arr['profit']=(profit_arr['daily_ret']+1).cumprod()
daily_profit=profit_arr['daily_ret'].tolist()
cumulative=profit_arr['profit'].dropna().tolist()
#print(daily_profit)
profit_arr['yr_month']=rank1.yr_month
profit_arr.to_csv('multi_3factor_profit_arr.csv')
win_arr = [x for x in daily_profit if x > 0]
loss_arr=[x for x in daily_profit if x < 0]
win_rate = len(win_arr) / (len(loss_arr)+len(win_arr))
#cumulative.to_csv('3factor_cumulative.csv')
dd_arr=[]
for i in range(len(cumulative)):
    if i>0:
        dd_arr.append((max(cumulative[:i])- cumulative[i-1])/max(cumulative[:i]))
max_dd=max(dd_arr)
print('weight1:{} and weight2:{}'.format(weight1,weight2))
print('CUMULATIVE PROFIT: {}'.format(round(cumulative[-1],4)))
print('MDD: {}' .format(max_dd))
print("mm_holding:{}------winrate:{}-----profit/MDD:{}".format(mm_hold, round(win_rate,4), round(cumulative[-1]/max_dd,4)))

fig = go.Figure()
fig.add_trace(go.Scatter(x=profit_arr['yr_month'].astype(str).tolist(), y=profit_arr['profit'].tolist(),
                             mode='lines',
                             name='cumulative'))

fig.show()



weight1:1 and weight2:0.0
CUMULATIVE PROFIT: 13.3454
MDD: 0.38681949724596426
mm_holding:10------winrate:0.7051-----profit/MDD:34.5004


Calculation of profit based on generated signal
